In [2]:
import duckdb
import pandas as pd
import numpy as np
import sys
import re
sys.path.insert(0, "../modules")
import plotly.io as pio
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import cufflinks as cf
cf.go_offline()

In [31]:
def row_filter(df, cat_var, cat_values):
    df = df[df[cat_var].isin([cat_values])].sort_values(by='Name', ascending=False)
    return df.reset_index(drop=True)

def nrow_filter(df, cat_var, cat_values):
    df = df[~df[cat_var].isin([cat_values])].sort_values(by='price', ascending=False)
    return df.reset_index(drop=True)

def groupby_sum(df, group_vars, agg_var='price', sort_var='price'):
    df = df.groupby(group_vars, as_index=False).agg({agg_var:'sum'})
    df = df.sort_values(by=sort_var, ascending=False)
    return df.reset_index(drop=True)

def pivot_table(df, col, x_axis, value='price'):
    df = df.pivot_table(values=value,
                        columns=col,
                        index=x_axis,
                        aggfunc='sum')
    return df.reset_index()

def groupby_median(df, group_vars, agg_var='price/carat', sort_var='price/carat'):
    df = df.groupby(group_vars, as_index=False).agg({agg_var:'median'})
    df = df.sort_values(by=sort_var, ascending=False)
    return df.reset_index(drop=True)

def groupby_mean(df, group_vars, agg_var='price/carat', sort_var='price/carat'):
    df = df.groupby(group_vars, as_index=False).agg({agg_var:'mean'})
    df = df.sort_values(by=sort_var, ascending=False)
    return df.reset_index(drop=True)

In [22]:
salaries = pd.read_csv('../data/Current_Employee_Names__Salaries__and_Position_Titles.csv')
salaries

,Name,Job Titles,Department,Full or Part-Time,Salary or Hourly,Typical Hours,Annual Salary,Hourly Rate
0,"AARON, JEFFERY M",SERGEANT,POLICE,F,Salary,NaN,101442.0,NaN
1,"AARON, KARINA",POLICE OFFICER (ASSIGNED AS DETECTIVE),POLICE,F,Salary,NaN,94122.0,NaN
2,"AARON, KIMBERLEI R",CHIEF CONTRACT EXPEDITER,GENERAL SERVICES,F,Salary,NaN,101592.0,NaN
3,"ABAD JR, VICENTE M",CIVIL ENGINEER IV,WATER MGMNT,F,Salary,NaN,110064.0,NaN
4,"ABASCAL, REECE E",TRAFFIC CONTROL AIDE-HOURLY,OEMC,P,Hourly,20.0,NaN,19.86
...,...,...,...,...,...,...,...,...
33178,"ZYLINSKA, KATARZYNA",POLICE OFFICER,POLICE,F,Salary,NaN,72510.0,NaN
33179,"ZYMANTAS, LAURA C",POLICE OFFICER,POLICE,F,Salary,NaN,48078.0,NaN
33180,"ZYMANTAS, MARK E",POLICE OFFICER,POLICE,F,Salary,NaN,90024.0,NaN
33181,"ZYRKOWSKI, CARLO E",POLICE OFFICER,POLICE,F,Salary,NaN,93354.0,NaN


In [21]:
salaries['Department'].unique()

array(['POLICE', 'GENERAL SERVICES', 'WATER MGMNT', 'OEMC',
       'CITY COUNCIL', 'AVIATION', 'STREETS & SAN', 'FIRE',
       'FAMILY & SUPPORT', 'PUBLIC LIBRARY', 'TRANSPORTN',
       "MAYOR'S OFFICE", 'HEALTH', 'BUSINESS AFFAIRS', 'LAW', 'FINANCE',
       'CULTURAL AFFAIRS', 'COMMUNITY DEVELOPMENT', 'PROCUREMENT',
       'BUILDINGS', 'ANIMAL CONTRL', 'CITY CLERK', 'BOARD OF ELECTION',
       'DISABILITIES', 'HUMAN RESOURCES', 'DoIT', 'BUDGET & MGMT',
       'TREASURER', 'INSPECTOR GEN', 'HUMAN RELATIONS', 'COPA',
       'BOARD OF ETHICS', 'POLICE BOARD', 'ADMIN HEARNG',
       'LICENSE APPL COMM'], dtype=object)

In [24]:
salaries['Full or Part-Time'].unique()

array(['F', 'P'], dtype=object)

In [23]:
salaries['Job Titles'].unique()

array(['SERGEANT', 'POLICE OFFICER (ASSIGNED AS DETECTIVE)',
       'CHIEF CONTRACT EXPEDITER', ..., 'CHIEF WATER ENGINEER',
       'COORD OF PRINTING SERVICES - GRAPHICS', 'MECHANICAL ENGINEER IV'],
      dtype=object)

In [16]:
salaries.describe()

,Typical Hours,Annual Salary,Hourly Rate
count,8022.000000,25161.000000,8022.000000
mean,34.507604,86786.999790,32.788558
std,9.252077,21041.354602,12.112573
min,10.000000,7200.000000,2.650000
25%,20.000000,76266.000000,21.200000
50%,40.000000,90024.000000,35.600000
75%,40.000000,96060.000000,40.200000
max,40.000000,300000.000000,109.000000


In [17]:
salaries.median()

C:\Users\marym\AppData\Local\Temp\ipykernel_23432\4231297940.py:1: FutureWarning:

The default value of numeric_only in DataFrame.median is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.



Typical Hours       40.0
Annual Salary    90024.0
Hourly Rate         35.6
dtype: float64

In [25]:
salaries.corr()

C:\Users\marym\AppData\Local\Temp\ipykernel_23432\3094442422.py:1: FutureWarning:

The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.



,Typical Hours,Annual Salary,Hourly Rate
Typical Hours,1.000000,NaN,0.738077
Annual Salary,NaN,1.0,NaN
Hourly Rate,0.738077,NaN,1.000000


Bins

In [ ]:
Proportions

In [27]:
salaries['Department'].value_counts(normalize= True)

POLICE                   0.404243
FIRE                     0.139861
STREETS & SAN            0.066239
OEMC                     0.063346
WATER MGMNT              0.056625
AVIATION                 0.049091
TRANSPORTN               0.034355
PUBLIC LIBRARY           0.030588
GENERAL SERVICES         0.029533
FAMILY & SUPPORT         0.018534
FINANCE                  0.016876
HEALTH                   0.014706
CITY COUNCIL             0.012386
LAW                      0.012265
BUILDINGS                0.008107
COMMUNITY DEVELOPMENT    0.006238
BUSINESS AFFAIRS         0.005153
COPA                     0.003496
BOARD OF ELECTION        0.003225
DoIT                     0.002983
PROCUREMENT              0.002773
INSPECTOR GEN            0.002622
MAYOR'S OFFICE           0.002562
CITY CLERK               0.002531
ANIMAL CONTRL            0.002441
HUMAN RESOURCES          0.002381
CULTURAL AFFAIRS         0.001959
BUDGET & MGMT            0.001386
ADMIN HEARNG             0.001175
DISABILITIES  

In [28]:
salaries['Job Titles'].value_counts(normalize= True)

POLICE OFFICER                            0.286894
FIREFIGHTER-EMT                           0.044390
SERGEANT                                  0.036223
POOL MOTOR TRUCK DRIVER                   0.031793
POLICE OFFICER (ASSIGNED AS DETECTIVE)    0.029804
                                            ...   
INQUIRY AIDE I                            0.000030
SECRETARY TO THE CHAIRMAN                 0.000030
CHIEF CONSTRUCTION EQUIPMENT INSPECTOR    0.000030
CLAIMS MANAGER                            0.000030
MECHANICAL ENGINEER IV                    0.000030
Name: Job Titles, Length: 1111, dtype: float64

In [26]:
salaries['Full or Part-Time'].value_counts(normalize= True)

F    0.936926
P    0.063074
Name: Full or Part-Time, dtype: float64

Separate Full or Hourly

In [ ]:
def row_filter(df, cat_var, cat_values):
    df = df[df[cat_var].isin([cat_values])].sort_values(by='price', ascending=False)
    return df.reset_index(drop=True)

In [34]:
salaries_F = row_filter(salaries, 'Full or Part-Time', 'F')
salaries_F['Salary or Hourly'].unique()

array(['Salary', 'Hourly'], dtype=object)

In [35]:
salaries_P = row_filter(salaries, 'Full or Part-Time', 'P')
salaries_P['Salary or Hourly'].unique()

array(['Hourly', 'Salary'], dtype=object)